In [1]:
import DRLPDE_main
import time

In [2]:
start = time.time()
DRLPDE_main.maintraining(param='JCPexample3' , use_cuda=True)

end = time.time()
print('Time to train: {:3.1f} seconds'.format(end - start))

Pre-processing: Loading parameters from JCPexample3.py
Initializing the model
Training has begun
No errors in first epoch: Training will continue
step = 500/5000, 0.242 s/step, time-to-go:1091s
step = 1000/5000, 0.254 s/step, time-to-go:1018s
step = 1500/5000, 0.261 s/step, time-to-go:912s
step = 2000/5000, 0.265 s/step, time-to-go:796s
step = 2500/5000, 0.269 s/step, time-to-go:672s
step = 3000/5000, 0.271 s/step, time-to-go:543s
step = 3500/5000, 0.273 s/step, time-to-go:409s
step = 4000/5000, 0.274 s/step, time-to-go:274s
step = 4500/5000, 0.275 s/step, time-to-go:137s
step = 5000/5000, 0.275 s/step, time-to-go: 0s
model saved as savedmodels/JCPexample1.pt
Time to train: 1375.4 seconds


In [ ]:
import DRLPDE_postprocessing
DRLPDE_postprocessing.postprocessing(param='JCPexample2',use_model='JCPexample1')

True